# Nyílt kulcsú titkosítás (RSA) / public key cryptography (RSA)

In [ ]:
"""
This program implements the RSA algorithm for cryptography.
It randomly selects two prime numbers and uses them to produce 
the public and private keys. Using the keys, it can 
either encrypt or decrypt messages.

Based on:
    https://github.com/jchen2186/rsa-implementation/blob/master/rsa.py
    https://stackoverflow.com/questions/567222/simple-prime-generator-in-python
"""
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import random, binascii, textwrap

In [ ]:
@interact(a=(1,1000), b=(1,1000))
def gcd(a, b):
    """
    Performs the Euclidean algorithm and returns the greatest common divisor of a and b
    """
    if (b == 0):
        return a
    else:
        return gcd(b, a % b)

In [ ]:
@interact(a=(1,1000), b=(1,1000))
def xgcd(a, b):
    """
    Performs the extended Euclidean algorithm
    Returns the gcd, coefficient of a, and coefficient of b
    """
    x, old_x = 0, 1
    y, old_y = 1, 0

    while (b != 0):
        quotient = a // b
        a, b = b, a - quotient * b
        old_x, x = x, old_x - quotient * x
        old_y, y = y, old_y - quotient * y

    return a, old_x, old_y

In [ ]:
@interact_manual(totient=(1000,10000,1000))
def chooseE(totient):
    """
    Chooses a random number, 1 < e < totient, and checks whether or not it is 
    coprime with the totient, that is, gcd(e, totient) = 1
    """
    while (True):
        e = random.randrange(2, totient)

        if (gcd(e, totient) == 1):
            return e

In [ ]:
@interact_manual(num=(1,10000))
def is_prime(num):
    """
    Returns True if the number is prime else False.
    """
    if num == 0 or num == 1:
        return False
    for x in range(2, num):
        if num % x == 0:
            return False
    else:
        return True

In [ ]:
@interact_manual(min=1000, max=10000)
def choosePrime(min, max):
    """
    Chooses a random number, min < p < max, and checks whether or not it is prime
    """
    while (True):
        p = random.randrange(min, max)

        if is_prime(p):
            return p

In [ ]:
@interact_manual(magnitude=(1,7))
def chooseKeys(magnitude = 5):
    """
    Selects two random prime numbers. Using the prime numbers, 
    it also computes the public and private keys.
    magnitude selects order of magnitude for the primes
    """

    # store our prime numbers in these variables
    prime1 = choosePrime(pow(10,magnitude), pow(10,magnitude+1))
    prime2 = choosePrime(pow(10,magnitude), pow(10,magnitude+1))

    # compute n, totient, e
    n = prime1 * prime2
    totient = (prime1 - 1) * (prime2 - 1)
    e = chooseE(totient)

    # compute d, 1 < d < totient such that ed = 1 (mod totient)
    # e and d are inverses (mod totient)
    gcd, x, y = xgcd(e, totient)

    # make sure d is positive
    if (x < 0):
        d = x + totient
    else:
        d = x

    return ([n, e], [n, d])


In [ ]:
(public, private) = chooseKeys(7)
display(public, private)

In [ ]:
@interact(text="plaintext", block_size=(1,10))
def text_to_numbers(text, block_size=2):
    """
    Helper function that splits up text into block_size chunks
    then converts those to integer values.
    Returns a string of integers
    """
    res = []
    for t in textwrap.wrap(text, block_size):
        res.append(str(int(binascii.hexlify(t.encode()), 16)))
    return ','.join(res)

In [ ]:
@interact(numbers=text_to_numbers("plaintext",2))
def numbers_to_text(numbers):
    """
    Helper function to invert effect of text_to_numbers()
    """
    res = ""
    for n in numbers.split(','):
        res += binascii.unhexlify(hex(int(n))[2:]).decode()
    return res

In [ ]:
@interact_manual(message="plaintext", key={'public': public, 'private': private}, block_size=(1,6))
def encrypt(message, key, block_size = 2):
    """
    Encrypts a message (string) by raising each characterblock's value to the 
    power of e and taking the modulus of n. Returns a string of numbers.
    block_size refers to how many characters make up one group of numbers in 
    each index of encrypted_blocks.
    """

    n = key[0]
    e = key[1]
    
    blocks = text_to_numbers(message, block_size).split(",")
    encrypted_blocks = []

    # encrypt all of the numbers by taking them to the power of e
    # and modding them by n
    for i in range(len(blocks)):
        num = int(blocks[i])
        if num > n:
            raise Exception('block size too large for key')
        encrypted_blocks.append(str(pow(num,e,n)))

    # create a string from the numbers
    encrypted_message = ",".join(encrypted_blocks)

    return encrypted_message


In [ ]:
@interact_manual(blocks=encrypt("plaintext", public, 2), key={'private': private, 'public': public})
def decrypt(blocks, key):
    """
    Decrypts a string of numbers by raising each number to the power of d and 
    taking the modulus of n. Returns the message as a string.
    """
    n = key[0]
    d = key[1]

    # turns the string into a list of ints
    list_blocks = blocks.split(',')
    int_blocks = []

    for s in list_blocks:
        int_blocks.append(int(s))

    decrypted_blocks = []
            
    # converts each int in the list of characters
    for i in range(len(int_blocks)):
        # decrypt all of the numbers by taking them to the power of d
        # and modding them by n
        decrypted_blocks.append(str(pow(int_blocks[i],d,n)))       

    return numbers_to_text(",".join(decrypted_blocks))